In [ ]:
!pip install requests beautifulsoup4 sentence-transformers faiss-cpu transformers accelerate huggingface_hub bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
 

In [ ]:
pip install accelerate bitsandbytes

In [ ]:
pip install bitsandbytes


In [ ]:
!pip install flask
!pip install pyngrok
!pip install flask_ngrok


In [ ]:
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import re
from sklearn.feature_extraction.text import CountVectorizer
from huggingface_hub import login


In [ ]:
login("hf_luDSzQSMnTlQvnttQlAEOOEWFMUYRMgOW")  # Replace with your Hugging Face API tokenx


In [ ]:
#thisis gpu code
# Define the quantization configuration
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the LLaMA-2 model and tokenizer with the specified quantization config
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
llama_model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=quantization_config)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
def scrape_webpage(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        page_text = ' '.join([p.get_text() for p in soup.find_all(['p', 'li', 'h1', 'h2', 'h3'])])
        return page_text
    else:
        return None


In [ ]:
def chunk_text(text, max_length=500):
    sentences = re.split(r'(?<=[.!?]) +', text)
    chunks = []
    current_chunk = []
    current_length = 0
    for sentence in sentences:
        if current_length + len(sentence) > max_length:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_length = 0
        current_chunk.append(sentence)
        current_length += len(sentence)
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks


In [ ]:
# embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')  # Load on CPU to save GPU memory

def index_chunks(chunks):
    embeddings = embedding_model.encode(chunks, device='cpu')
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(np.array(embeddings))
    return index, chunks


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
summary_examples = [
    "summarize this article",
    "give an overview",
    "what is this article about",
    "what are the main points",
    "tell me about this article",
    "provide a summary",
    "explain the content briefly"
]

# Encode  questions
summary_example_embeddings = embedding_model.encode(summary_examples)

def is_summary_question(question, threshold=0.7):
    question_embedding = embedding_model.encode([question])[0]
    similarities = np.dot(summary_example_embeddings, question_embedding) / (
        np.linalg.norm(summary_example_embeddings, axis=1) * np.linalg.norm(question_embedding)
    )
    return np.max(similarities) > threshold


In [ ]:
import re

def summarize_article(chunks):
    print("Summarizing started")

    def clean_text(text):
        text = re.sub(r'\b(US|International|Arabic|Español|World|Settings|Topics You Follow|Sign Out|CNN Audio podcasts|More News|Related article|CNN 5 Things|Latest Market News|Listen|Live TV|CNN TV Schedules|Africa|Americas|Asia|Europe|Middle East|United Kingdom|Health|Entertainment|Tech|Style|Travel|Sports|Watch)\b', '', text)
        text = re.sub(r'\s+', ' ', text)
        return text.strip()

    filtered_chunks = [clean_text(chunk) for chunk in chunks if len(clean_text(chunk).split()) > 10]
    max_paragraphs = 4
    selected_chunks = filtered_chunks[:max_paragraphs]
    full_text = " ".join(selected_chunks)

    prompt = f"Summarize this article content concisely:\n\n{full_text}\n\n what is this article about?"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = llama_model.generate(**inputs, max_new_tokens=200)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    print("Pre-summary output:\n", answer)


    if answer and isinstance(answer, str):
        match = re.search(r'what is this article about\?\s*(.*)', answer, re.DOTALL | re.IGNORECASE)
        if match:
            extracted_answer = match.group(1).strip()
        else:
            extracted_answer = "Unable to extract a clear summary. The expected format was not found."
    else:
        extracted_answer = "No valid answer returned from the model."

    print("Extracted summary:\n", extracted_answer)
    return extracted_answer


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def is_relevant(context, question, threshold=0.3):

    context_embedding = embedding_model.encode([context])
    question_embedding = embedding_model.encode([question])


    similarity_score = cosine_similarity(context_embedding, question_embedding)[0][0]
    return similarity_score > threshold  # Return True if above threshold


In [ ]:
def clean_context(text):

    cleaned_text = re.sub(r'\s+', ' ', text).strip()
    return cleaned_text


def retrieve_and_answer(question, index, chunks):
    torch.cuda.empty_cache()


    if is_summary_question(question):
        return summarize_article(chunks)


    question_embedding = embedding_model.encode([question])
    _, indices = index.search(np.array(question_embedding), k=3)
    context = " ".join([chunks[i] for i in indices[0]])


    context = clean_context(context)
    print("\nRetrieved Context:\n", context)


    if not is_relevant(context, question):
        return "I don't have an answer for this in my knowledge base."


    prompt = f"Provide a detailed answer based on the context below:\n\nContext: {context}\n\nQuestion: {question}"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = llama_model.generate(**inputs, max_new_tokens=100)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    return answer


In [ ]:
!ngrok config add-authtoken '2t5jG4Kb85xObYFT6mzyG088vu1_61sQEm5o3GesZbmChuQH'
#add ngrock tokenR

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("2t5jG4Kb85xObYFT6mzyG088vu1_61sQEm5o3GesZbmChuQH")


In [ ]:
from googlesearch import search
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)

def search_and_scrape(query, num_results=3):
    try:
        search_results = search(query)
        if not search_results:
            logging.error("No search results found.")
            return "Error: No search results found."

        all_content = ""

        for i, url in enumerate(search_results):
            if i >= num_results:
                break
            logging.info(f"Scraping content from {url}")
            content = scrape_webpage(url)

            # Check if content is None or not a string
            if content and isinstance(content, str) and not content.startswith("Error"):
                all_content += content + " "
            else:
                logging.error(f"Failed to retrieve or process content from {url}")

        return all_content if all_content.strip() else "Error: No content found from search results."
    except Exception as e:
        logging.error(f"Error during Google search: {str(e)}")
        return f"Error during Google search: {str(e)}"


In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok


public_url = ngrok.connect(5000)
print("Ngrok tunnel URL:", public_url.public_url)

app = Flask(__name__)
run_with_ngrok(app)


app_index = None
app_chunks = None


@app.route('/process', methods=['POST'])
def process_url():
    data = request.json
    url = data.get('url')


    page_text = scrape_webpage(url)
    if not page_text:
        return jsonify({"error": "Failed to retrieve content from the URL."}), 400


    chunks = chunk_text(page_text)
    index, indexed_chunks = index_chunks(chunks)


    global app_index, app_chunks
    app_index, app_chunks = index, indexed_chunks


    return jsonify({"success": True})

@app.route('/ask', methods=['POST'])
def ask_question():
    data = request.json
    question = data.get('question')

    if not app_index or not app_chunks:
        return jsonify({"error": "No indexed content available. Please process a URL first."}), 400

    answer = retrieve_and_answer(question, app_index, app_chunks)


    match = re.search(r'(?:Answer:\s*|Question:[^\n]*\n\n)(.*)', answer, re.DOTALL)
    if match:
      extracted_answer = match.group(1).strip()
      print("Extracted Answer:", extracted_answer)
      return jsonify({"response": extracted_answer})
    else:
      return jsonify({"response": answer})
@app.route('/search', methods=['POST'])
def search_and_respond():
    global vector_store
    data = request.get_json()
    query = data.get('query')

    search_content = search_and_scrape(query)
    if search_content.startswith("Error"):
        return jsonify({"error": search_content}), 500

    chunks = chunk_text(search_content)
    index, indexed_chunks = index_chunks(chunks)
    app_index, app_chunks = index, indexed_chunks
    answer = retrieve_and_answer(query, app_index, app_chunks)
    match = re.search(r'(?:Answer:\s*|Question:[^\n]*\n\n)(.*)', answer, re.DOTALL)
    if match:
      extracted_answer = match.group(1).strip()
      print("Extracted Answer:", extracted_answer)
      return jsonify({"success": True,"response": extracted_answer})
    else:
      return jsonify({"success": True, "response": answer})
    # return jsonify({"success": True, "response": answer})

if __name__ == '__main__':
    app.run()


Ngrok tunnel URL: https://5f19-34-16-186-250.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://5f19-34-16-186-250.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [19/Mar/2025 06:12:24] "POST /process HTTP/1.1" 200 -


Summarizing started


...

Pre-summary output:
 Summarize this article content concisely:

December 28, 2024e-Paper The Hindu On Books Books of the week, reviews, excerpts, new titles and features. Data Point Decoding the headlines with facts, figures, and numbers First Day First Show News and reviews from the world of cinema and streaming. Matters Ramya Kannan writes to you on getting to good health, and staying there The View From India Looking at Affairs from the Indian perspective. Science For All The weekly newsletter from science writers takes the jargon out of science and puts the fun in! Karnataka Today Your daily dose of news highlights from Karnataka Today's Cache Your download of the top 5 technology stories of the day. December 28, 2024e-Paper Home News India Understanding the delimitation exercise | Explained What does delimitation mean and what are the constitutional provisions which deal with it? Why were the seats frozen as per the 1971 Census? Will the fresh exercise go against the federal princ

ERROR:root:Failed to retrieve or process content from https://jair.org/index.php/jair/article/view/11315
ERROR:root:Failed to retrieve or process content from https://brainly.com/question/10633645


Summarizing started


INFO:werkzeug:127.0.0.1 - - [19/Mar/2025 06:13:20] "POST /search HTTP/1.1" 200 -


Pre-summary output:
 Summarize this article content concisely:

Chat with PDF Browser Extensions Features Pricing Products Sign in Sign Up Chat with PDF Browser Extensions Features ProductsChatsonicWritesonicBotsonic Chatsonic Writesonic Botsonic Sign in Sign Up Summarize any | in a click. TLDR This helps you summarize any piece of text into concise, easy to digest content so you can free yourself from information overload. Trusted By Enter an Article URL or paste your Text Browser Extensions Use TLDR This browser extensions to summarize any webpage in a click. Single platform, endless summaries Transforming information overload into manageable insights — consistently striving for clarity. 100% Automatic Article Summarization with just a click In the sheer amount of information that bombards Internet users from all sides, hardly anyone wants to devote their valuable time to reading long texts. TLDR This's clever AI analyzes any piece of text and summarizes it automatically, in a way th